# kinetic proofreading animation
PCN 5/2018<br>
This code just displays the results from riboproof2.py, via the data file kproof.npz<br>
tRNA-AA-GTP complexes appear suddenly from solution<br>
red = wrong; green = right<br>
GTP = dark color; GDP = bright color<br>
GTP = cylinder; GDP = sphere; <br>
GTP hydrolysis = shift right and deepening color<br>
rejected tRNA-AA-GxP appear briefly ghosted then disappear<br>
AA incorporation shifts the nascent chain over to the right.

The configuration shown is the one current at the end of each video frame. If initial binding is rapidly followed by hydrolysis in the same video frame, then the binding will not be displayed. If hyrolysis is rapidly followed by rejection or incorporation, then the hydrolysis will not be shown.

Suggested: duration=10000 for a very long and detailed look.
duration=1200 for a less detailed overview

In [ ]:
debug = False                # whether to dump reports into a file

from vpython import *
from numpy.random import random as rng
import numpy as np
fracshow = 0.95             # stop slightly short of end (e.g. 0.94)
framerate = 15              # frame rate in Hz, so 30 means 33ms per frame
duration = 10000            # desired duration of movie [s]
mysphererad = 0.15          # blob size representing amino acids
hydrolysis_sh = mysphererad-.05
data = np.load('kproof.npz')# generated by simulation engine riboproof2.py
ts = data['ts'][0]          # transition times
states = data['states'][0]  # sequence of states:  states[i] is state exited at ts[i]. 
                            # states[i+1] is state entered at ts[i].
Nstates = len(states)
timeunit = duration/ts[-1]  # real time [s] per unit in the simulation
rejectduration = 5          # how long to display rejected (in frames)
# how many frames we'll see
print("Planning to show ",framerate*duration*fracshow," frames over ",duration," seconds.")

In [ ]:
if debug: debugfile = open('debug.txt', 'w')
scene = canvas(title='KP')
scene.forward = vector(0,.4,-1) # camera viewpoint
# represent ribosome by a blue tabletop
static = box(pos=vector(0,0,0), size=vector(1,1,.2), color=color.blue)
bindsite = vector(-.15,0,.1)
# represent the bound tRNA-GTP complex by a cylinder: invisible if none bound, dark
#  green if right, dark red if wrong
boundT = cylinder(pos=bindsite+vector(0,0,mysphererad*1.5), axis=vector(0,0,.17),\
                  radius=mysphererad*.85)
boundT.visible = False # initially none
# after a tRNA-GTP is rejected, briefly show it as ghost
rejectT = cylinder(pos=bindsite+vector(.5*mysphererad,0,mysphererad*1.5), axis=vector(0,0,.17),\
                   radius=mysphererad*.85, opacity=0.2)
rejectT.visible = False
# represent bound tRNA-GDP complex by a sphere, with bright color
boundD = sphere(pos=bindsite+vector(hydrolysis_sh,0,mysphererad*2), radius=mysphererad)
boundD.visible = False # initially none
# after a tRNA-GDP is rejected, briefly show it as ghost
rejectD = sphere(pos=bindsite+vector(.5*mysphererad+hydrolysis_sh,0,mysphererad*2), \
                 radius=mysphererad, opacity=0.2)
rejectD.visible = False
# just  before a tRNA-GTP binds, briefly show it as ghost
arrivingT = cylinder(pos=bindsite+vector(-.5*mysphererad, 0, mysphererad*(1.5)), \
                     axis=vector(0,0,.17), radius=mysphererad*.85, opacity=0.2)
arrivingT.visible = False

chain = [] #list of blobs representing incorporated AAs. Last item in chain is most recent 
           #addition and will be displayed leftmost.
chainsite = vector(+.25,0,.1)
timerT = 0; timerD = 0      # initialize
frametext = label(text=str(0), color=color.green)

State is initially states[0]=0. At time ts[i] transition from states[i] to states[i+1]. Note states[i] is negative if this exit involves an amino acid incorporation.<br>
In each video frame, show the state that is current (next to be exited) at the end of that frame, along with any arrivals or rejections that may have occurred during that interval.<br>
Warning: VPython may skip frames. To avoid this, (1) stop other CPU-using processes first and close other resource-hogging browser windows. (2) choose a slower value for framerate. (3) turn off debug=False. (4) do a video capture of the animation and step through it to see if frames are dropped.

In [ ]:
j=0  # which event we're on now
for whichframe in range(int(framerate*duration*fracshow)):
    timenow = whichframe/framerate           # time at which this video frame ends
    frametext.text = str(whichframe)+": {:.2f}".format(timenow) # update display
    if debug: debugfile.write("Frame "+str(whichframe)+" first exits state j="+str(j)+\
                              " which is "+str(states[j])+"; timenow="+str(timenow)+'\n')
    arriving = 0; arrivingT.visible = False  # reset the arriving amino on each video frame
    # process all events that happened since last video frame:
    while ts[j]*timeunit < timenow:          # did any transition j occur before end of the current
                                             # video frame that hasn't yet been processed?
        # yes, so first exit state j:
        if states[j]==-3:                    # elongate chain: correct incorporation
            # this is not a rejection so don't turn on rejectD, but if it's already on from a
            # previous rejection event leave it on
            for amino in chain:              # push everybody to right
                amino.pos = amino.pos + vector(mysphererad*1.9,0,0)
            chain = chain + [sphere(pos=chainsite+vector(0,0,.3), \
                                    radius=mysphererad,color=vector(0,1,0))]
            print('correct incorporation frame ',whichframe,', j=', j)
        elif states[j]==-4:                  # elongate chain: wrong incorporation
            # this is not a rejection so don't turn on rejectD, but if it's already on from a
            # previous rejection event leave it on
            for amino in chain:              # push everybody to right
                amino.pos = amino.pos + vector(mysphererad*2,0,0)
            chain = chain + [sphere(pos=chainsite+vector(0,0,.3), \
                                    radius=mysphererad,color=vector(1,0,0))]
            print('wrong incorporation frame ',whichframe,', j=', j)
        j+=1 #now enter next state. If any arrival or rejection happens in any event of this 
             #video frame, show it 
        if debug: debugfile.write('then exits state j='+str(j)+" which is "+str(states[j])+'\n')
        if states[j]==0: 
            if states[j+1]==1: arriving = 1      # a correct will bind on next step
            if states[j+1]==2: arriving = 2      # an incorrect will bind on next step
            if states[j-1]==1:  # correct.T unbinds: dark translucent green
                rejectT.visible = True           #later will also turn off boundT
                timerT = 0                       # display rejected amino for a while 
                rejectT.color = vector(.5,1,.5)
                rejectT.pos = bindsite+vector(.5*mysphererad,0,mysphererad*1.5)
            elif states[j-1]==2:# wrong.T unbinds: dark translucent red
                rejectT.visible = True           #later will also turn off boundT
                timerT = 0                       # display rejected amino for a while 
                rejectT.color = vector(1,.5,.5)
                rejectT.pos = bindsite+vector(.5*mysphererad,0,mysphererad*1.5)
            elif states[j-1]==3:# correct.D unbinds: bright translucent green
                rejectD.visible = True           #later will also turn off boundD
                timerD = 0                       # display rejected amino for a while 
                rejectD.color = vector(0,1,0)
                rejectD.pos = bindsite+vector(.5*mysphererad+hydrolysis_sh,0,mysphererad*2)
            elif states[j-1]==4:# wrong.D unbinds: bright translucent red
                rejectD.visible = True           #later will also turn off boundD
                timerD = 0                       # display rejected amino for a while 
                rejectD.color = vector(1,0,0)
                rejectD.pos = bindsite+vector(.5*mysphererad+hydrolysis_sh,0,mysphererad*2)
    #now show whatever state we're in at the start of new video frame
    newstate = states[j]
    if newstate==0:                        # unoccupied; continue to display rejected if there is one
        boundT.visible = False
        boundD.visible = False
    elif newstate==1:                      # correct tRNA.GTP: dark green
        boundT.visible = True
        boundT.color = vector(0,.3,0)
        boundD.visible = False
    elif newstate==2:                      # wrong tRNA.GTP: dark red
        boundT.visible = True
        boundT.color = vector(.3,0,0)
        boundD.visible = False
    elif newstate==3 or newstate==-3:      # correct tRNA.GDP: green
        boundT.visible = False
        boundD.visible = True
        boundD.color = vector(0,1,0)
    elif newstate==4 or newstate==-4:      # wrong tRNA.GDP: red
        boundT.visible = False
        boundD.visible = True
        boundD.color = vector(1,0,0)
    # animate rejection:
    if timerD > rejectduration: rejectD.visible = False
    else: rejectD.pos += vector(0,0,.15*mysphererad*1.5)
    if timerT > rejectduration: rejectT.visible = False
    else: rejectT.pos += vector(0,0,.15*mysphererad*1.5)
    if debug: 
        debugfile.write('rejectD '+str(rejectD.visible)+'\n')
        debugfile.write('rejectT '+str(rejectT.visible)+'\n')
    timerT += 1; timerD += 1
    # animate arrival:
    if arriving==1: 
        arrivingT.visible = True; arrivingT.color = vector(.5,1,.5)
    if arriving==2:
        arrivingT.visible = True; arrivingT.color = vector(1,.5,.5)
    if debug: debugfile.write('arriving '+str(arrivingT.visible)+'\n')
    rate(framerate)                          # pause till it's time to show next frame
if debug: debugfile.close()

In [ ]:
1